In [64]:
"""The pipes are arranged in a two-dimensional grid of tiles:

| is a vertical pipe connecting north and south.
- is a horizontal pipe connecting east and west.
L is a 90-degree bend connecting north and east.
J is a 90-degree bend connecting north and west.
7 is a 90-degree bend connecting south and west.
F is a 90-degree bend connecting south and east.
. is ground; there is no pipe in this tile.

S is the starting position; there is a pipe on this tile,
but the sketch doesn't show what shape the pipe has.
"""

example_1 =""".....
.F-7.
.|.|.
.L-J.
.....
"""

example_2 =""".....
.S-7.
.|.|.
.L-J.
.....
"""

example_2_1 ="""-L|F7
7S-7|
L|7||
-L-J|
L|-JF
"""

example_3 ="""..F7.
.FJ|.
SJ.L7
|F--J
LJ...
"""

example_3_1 ="""7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ
"""

In [65]:
import numpy as np
from collections import deque

# Define the possible pipe types
pipe_types = {
    '|': [(-1, 0), (1, 0)],
    '-': [(0, -1), (0, 1)],
    'L': [(0, -1), (1, 0)],
    'J': [(0, 1), (1, 0)],
    '7': [(0, 1), (-1, 0)],
    'F': [(0, -1), (-1, 0)],
    '.': [],
    'S': [(0, 1), (0, -1), (1, 0), (-1, 0)]
}

def split_input(input_data):
    return [list(row) for row in input_data.splitlines()]

def verify_symbol(mirrored_connections, local_grid_array, x_row, x_col):
    """Verify that the connections are valid for the given pipe type"""
    # Check each position relative to 'X'
    for dr, dc in mirrored_connections:
        new_row, new_col = x_row + dr, x_col + dc
        if 0 <= new_row < local_grid_array.shape[0] and 0 <= new_col < local_grid_array.shape[1]:
            if (dr,dc) not in pipe_types[local_grid_array[new_row, new_col][0]]:
                return None
    return mirrored_connections

def check_pipe(x_row, x_col, local_grid_array, original_char, debug=False):
    """
    Determine if a pipe is valid based on a 3x3 grid.
    Valid pipes have exactly two connections and is the expected
    character, or 'S'
    """
    # Check each position relative to 'X'
    valid_pipes = 0
    for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        new_row, new_col = x_row + dr, x_col + dc
        if debug:
            print(f"New row: {dr}, new col: {dc}, char: {local_grid_array[new_row, new_col]}")
        if 0 <= new_row < local_grid_array.shape[0] and 0 <= new_col < local_grid_array.shape[1]:
            if (dr,dc) in pipe_types[local_grid_array[new_row, new_col][0]]:
                if debug:
                    print(f"Found valid pipe {local_grid_array[new_row, new_col]}")
                valid_pipes += 1
    # If there's exactly two valid pipes next to 'X', return True, else False
    if valid_pipes == 2:
        if original_char != 'S':
            mirrored_connections = [(-dr, -dc) for dr, dc in pipe_types[original_char]]
            return verify_symbol(mirrored_connections, local_grid_array, x_row, x_col)
        else:
            # Try all possible combinations of connections
            for guess_char, mirrored_connections in pipe_types.items():
                mirrored_connections = [(-dr, -dc) for dr, dc in mirrored_connections]
                result = verify_symbol(mirrored_connections, local_grid_array, x_row, x_col)
                if result:
                    print(f"Assuming 'S' is {guess_char}")
                    return result

def shortest_path(pipes, start, end):
    # Create a queue for BFS
    queue = deque([(start, 0)])
    # Create a set to store visited nodes
    visited = set()
    while queue:
        node, steps = queue.popleft()
        # If this node is the destination, return the number of steps
        if node == end:
            return steps
        visited.add(node)
        for neighbor in pipes[node]:
            # If the neighbor has not been visited, enqueue it with the number of steps + 1
            if neighbor not in visited:
                queue.append((neighbor, steps + 1))
    # No path found
    return -1

content = np.array(split_input(example_2))
display(content)
# Loop through every character, take a 3x3 grid around it, and check if it's a valid pipe
valid_pipes = {}
for row in range(content.shape[0]):
    for col in range(content.shape[1]):
        debug = False
        # Handle edge cases by creating partial grids
        grid_slice = content[max(0, row-1):min(content.shape[0], row+2), max(0, col-1):min(content.shape[1], col+2)].copy()
        # Determine the position of 'X' in the grid slice
        x_row, x_col = min(1, row), min(1, col)
        # Save the original character and mark the current position with 'X'
        original_char = grid_slice[x_row, x_col]
        #if original_char == '-':
        #    debug = True
        grid_slice[x_row, x_col] = 'X'
        # print(grid_slice)
        neighbors = check_pipe(x_row, x_col,grid_slice, original_char,debug)
        if neighbors:
            valid_pipes[(row, col)] = [(row+dr, col+dc) for dr, dc in neighbors]
display(valid_pipes)
paths = [shortest_path(valid_pipes, (1, 1), target) for target in valid_pipes.keys()]
print(paths)
print(f"Max_length steps: {max(paths)}")

array([['.', '.', '.', '.', '.'],
       ['.', 'S', '-', '7', '.'],
       ['.', '|', '.', '|', '.'],
       ['.', 'L', '-', 'J', '.'],
       ['.', '.', '.', '.', '.']], dtype='<U1')

Assuming 'S' is F


{(1, 1): [(1, 2), (2, 1)],
 (1, 2): [(1, 3), (1, 1)],
 (1, 3): [(1, 2), (2, 3)],
 (2, 1): [(3, 1), (1, 1)],
 (2, 3): [(3, 3), (1, 3)],
 (3, 1): [(3, 2), (2, 1)],
 (3, 2): [(3, 3), (3, 1)],
 (3, 3): [(3, 2), (2, 3)]}

[0, 1, 2, 1, 3, 2, 3, 4]
Max_length steps: 4
